In [ ]:
import time
import os
import numpy as np
import tensorflow as tf
import importlib
from datetime import datetime
from tensorflow.python.framework.ops import reset_default_graph
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import utils

SAVER_PATH = {'base': 'train/',
              'checkpoint': 'checkpoints/',
              'log': 'logs/',
              'test': 'test/'}
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

def load_config(config_name):
    config_path = 'configurations.' + config_name
    config = importlib.import_module(config_path)
    return config

def load_path(config_name, split, epoch=None):
    name = "%s-%d" % (config_name, split)
    local_path = os.path.join(SAVER_PATH['base'], name)
    checkpoint_saver = tf.train.Saver()
    checkpoint_path = os.path.join(local_path, SAVER_PATH['checkpoint'])
    checkpoint_file_path = os.path.join(checkpoint_path, 'checkpoint')
    if epoch is None:
        latest_checkpoint = tf.train.latest_checkpoint(checkpoint_path)
    else:
        latest_checkpoint = "%s-%d" % (checkpoint_file_path, epoch)
    return checkpoint_saver, latest_checkpoint

def validate(sess, gen, l_out_softmax, X_pl, t_pl):
    outs = []
    targets = []
    v_sum = 0
    for batch, i in gen():
        fetches = [l_out_softmax]
        feed_dict = {X_pl: batch['X'], t_pl: batch['t'], is_training_pl: False}
        out = sess.run(fetches=fetches, feed_dict=feed_dict)[0]
        outs.append(out)
        targets.append(batch['t'])
        v_sum += i
    outs = np.concatenate(outs, axis=0)[:v_sum]
    targets = np.concatenate(targets, axis=0)[:v_sum]
    aucs = utils.auc(targets, outs[:, 1])
    preds = outs[:, 1]>0.5
    accs = np.mean(np.equal(preds, targets))
    return outs, targets, aucs, accs

In [ ]:
name_split_epochs = [("rnn_big", 1, [19501, 15001]),
                     ("rnn_big2", 1, [13501, 17501]),
                     ("rnn_big", 2, [27001, 19501]),
                     ("rnn_big2", 2, [17501, 23501]),
                     ("rnn_big", 3, [6501, 11001]),
                     ("rnn_big2", 3, [12501, 17501]),
                     ("rnn_big", 4, [12501, 8001]),
                     ("rnn_big2", 4, [18501, 8001]),
                     ("rnn_big", 5, [13001, 16001]),
                     ("rnn_big2", 5, [23501, 12501]),
                     ("rnn_big", 6, [25001, 20501]),
                     ("rnn_big2", 6, [16501, 22001]),
                     ("rnn_big", 7, [16001, 19501]),
                     ("rnn_big2", 7, [20001, 16001]),
                     ("rnn_big", 8, [22001, 26501]),
                     ("rnn_big2", 8, [22001, 16001])]

valid_preds = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}
valid_targets = {1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None}
test_preds = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}
test_targets = {1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None}
for config_name, split, epochs in name_split_epochs:
    print("%s-%d" % (config_name, split))
    for epoch in epochs:
        print("  %d" % epoch)
        reset_default_graph()
        config = load_config(config_name)
        data_gen = config.data_gen(split, train=False)
        X_pl, t_pl, is_training_pl, _, l_out_softmax, loss, accuracy, train_op, global_step = config.model()
        checkpoint_saver, latest_checkpoint = load_path(config_name, split, epoch=epoch)
        print(latest_checkpoint)
        gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
        with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
            checkpoint_saver.restore(sess, latest_checkpoint)
            outs, targets, aucs, accs = validate(sess, data_gen.gen_valid, l_out_softmax, X_pl, t_pl)
            print("  valid accs: %f" % accs)
            print("  valid aucs: %f" % aucs)
            valid_preds[split].append(outs)
            valid_targets[split] = targets
            outs, targets, aucs, accs = validate(sess, data_gen.gen_test, l_out_softmax, X_pl, t_pl)
            print("  test accs: %f" % accs)
            print("  test aucs: %f" % aucs)
            test_preds[split].append(outs)
            test_targets[split] = targets
    print

In [ ]:
valid_preds[3][0][:,1].mean()

In [ ]:
v_tot_preds = []
v_tot_targets = []
print("hi")
for key, values in valid_preds.iteritems():
    print("key,", key)
    print("values", len(values))
    tot_models = len(values)
    v_john = None
    for tot_out in values:
        if v_john is None:
            v_john = tot_out
        else:
            v_john += tot_out
    v_targets = valid_targets[key]
    v_aucs = utils.auc(v_targets, v_john[:, 1])
    v_preds = v_john[:, 1]>0.5
    v_accs = np.mean(np.equal(v_preds, v_targets))
    print("  valid accs: %f" % v_accs)
    print("  valid aucs: %f" % v_aucs)
    v_tot_preds.append(v_john)
    v_tot_targets.append(v_targets)
v_tot_preds = np.concatenate(v_tot_preds, axis=0)
v_tot_targets = np.concatenate(v_tot_targets, axis=0)
v_tot_aucs = utils.auc(v_tot_targets, v_tot_preds[:, 1])
v_tot_preds = v_tot_preds[:, 1]>0.5
v_tot_accs = np.mean(np.equal(v_tot_preds, v_tot_targets))
print("TOTAL VALID")
print("  valid accs: %f" % v_tot_accs)
print("  valid aucs: %f" % v_tot_aucs)

t_tot_preds = []
t_tot_targets = []
for key, values in test_preds.iteritems():
    print("key,", key)
    print("values", len(values))
    tot_models = len(values)
    t_john = None
    for tot_out in values:
        if t_john is None:
            t_john = tot_out
        else:
            t_john += tot_out
    t_john = t_john/float(tot_models)
    t_targets = test_targets[key]
    t_aucs = utils.auc(t_targets, t_john[:, 1])
    t_preds = t_john[:, 1]>0.5
    t_accs = np.mean(np.equal(t_preds, t_targets))
    print("  test accs: %f" % t_accs)
    print("  test aucs: %f" % t_aucs)
    t_tot_preds.append(t_john)
    t_tot_targets.append(t_targets)
t_tot_preds = np.concatenate(t_tot_preds, axis=0)
t_tot_targets = np.concatenate(t_tot_targets, axis=0)
t_tot_aucs = utils.auc(t_tot_targets, t_tot_preds[:, 1])
t_tot_preds = t_tot_preds[:, 1]>0.5
t_tot_accs = np.mean(np.equal(t_tot_preds, t_tot_targets))
print("TOTAL TEST")
print("  test accs: %f" % t_tot_accs)
print("  test aucs: %f" % t_tot_aucs)

In [ ]:
np.save("total_targets.npy", t_tot_targets)
np.save("total_predictions.npy", t_tot_preds)